In [1]:
import os
import requests
import time
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
query = """
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?company ?label
    WHERE {
        ?company wdt:P31/wdt:P279* wd:Q4830453;
                 rdfs:label ?label.
        FILTER(LANG(?label) = "en").
    }
    """

data = requests.get(url, params={'query': query, 'format': 'json'}).json()

In [3]:
companies = {}

for result in data["results"]["bindings"]:
    companies[result['company']['value'].split('/')[-1]] = result['label']['value']

In [4]:
company_aliases = {}

In [ ]:
for n, company in tqdm(enumerate(companies), total=len(companies)):
    if company not in company_aliases:
        print(n, company, companies[company])
        query = """
            PREFIX wikibase: <http://wikiba.se/ontology#>
            PREFIX wd: <http://www.wikidata.org/entity/>
            PREFIX wdt: <http://www.wikidata.org/prop/direct/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

            SELECT ?altLabel 
            WHERE {
                wd:$$company$$ skos:altLabel ?altLabel
                SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
            }
            """.replace("$$company$$", company)
        try:
            data = requests.get(url, params={'query': query, 'format': 'json'}).json()
            aliases = []
            for result in data["results"]["bindings"]:
                aliases.append(result['altLabel']['value'])

            company_aliases[company] = (companies[company], aliases)
        except:
            print("except")
            time.sleep(2)

0 Q11148 The Guardian
1 Q21202 Şalom
2 Q486314 The Hankyoreh
3 Q489214 Alaska Dispatch News
4 Q489308 Hankook Ilbo
5 Q489520 Korea JoongAng Daily
6 Q489650 Glas Koncila
7 Q492825 Daily NK
8 Q493265 Áššu
9 Q493319 Min Áigi
10 Q493500 The Namibian
except
11 Q13709 Le Canard enchaîné
12 Q13717 Libération
13 Q20580 Seelter Trjoue
except
14 Q262625 The Sowetan
15 Q265362 Der Freitag
16 Q269963 7Days
except
17 Q274581 Élet és Irodalom
18 Q277213 The Gazette of India
19 Q278151 Spiegel Online
except
20 Q279739 Business Line
21 Q283570 Moskauer Deutsche Zeitung
22 Q284947 The Christian Science Monitor
except
23 Q285738 Al-Hayat
24 Q147261 Zapf Umzüge
25 Q169356 ZebeMusic
except
26 Q198579 Polish Music Publishing House
27 Q203704 Zimmermann
28 Q294609 AMA Verlag
29 Q443654 Ernst Eulenburg
except
30 Q467407 G. Henle Verlag
31 Q468157 Edition Peters
32 Q524061 Wonderland Music Company, Inc.
33 Q536259 Schott Music
except
34 Q569779 Universal Edition
35 Q25727 Bitola Gazette
36 Q25738 Capital
exce

In [ ]:
len(companies)